In [4]:
import pandas as pd


In [5]:
df = pd.read_csv("Data/kitCapstoneJan24_data.csv", sep=";")

## selecting the country that we want

In [6]:
selected_country = 'Country A'
#Get rows with for country A

df_country_a = df[df['countryname'] == selected_country]
df_country_a

,month,countryname,sales_actuals_monthly__vehiclegroup01__orderintake,sales_actuals_monthly__vehiclegroup01__retail,sales_actuals_monthly__vehiclegroup02__orderintake,sales_actuals_monthly__vehiclegroup02__retail,sales_actuals_monthly__vehiclegroup03__orderintake,sales_actuals_monthly__vehiclegroup03__retail,sales_actuals_monthly__vehiclegroup04__orderintake,sales_actuals_monthly__vehiclegroup04__retail,...,registrations__buildtype_suv,registrations__buildtype_transport,registrations__buildtype_van,registrations__buildtype_cabrio,registrations__buildtype_stufenheck,registrations__buildtype_kurzheck,registrations__buildtype_other,registrations__buildtype_pickup,registrations__buildtype_coupe,registrations__buildtype_kombi
0,1,Country A,296.0,352.0,739.0,471.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Country A,414.0,372.0,904.0,528.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Country A,405.0,604.0,1000.0,953.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Country A,359.0,483.0,828.0,972.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Country A,403.0,474.0,594.0,900.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,356,Country A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
356,357,Country A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
357,358,Country A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
358,359,Country A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## building a additional feature for months (1-12)

important for meanbymonth

In [7]:
df_country_a['month_normalized'] = df_country_a['month'] % 12
df_country_a

df_country_a = df_country_a[['month_normalized','month', 'countryname'] + [col for col in df_country_a.columns if col not in ['month_normalized','month', 'countryname']]]

df_country_a

/var/folders/qs/1560hq_j7kl13jy4hgrfjtcc0000gn/T/ipykernel_68433/2443941320.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_country_a['month_normalized'] = df_country_a['month'] % 12


,month_normalized,month,countryname,sales_actuals_monthly__vehiclegroup01__orderintake,sales_actuals_monthly__vehiclegroup01__retail,sales_actuals_monthly__vehiclegroup02__orderintake,sales_actuals_monthly__vehiclegroup02__retail,sales_actuals_monthly__vehiclegroup03__orderintake,sales_actuals_monthly__vehiclegroup03__retail,sales_actuals_monthly__vehiclegroup04__orderintake,...,registrations__buildtype_suv,registrations__buildtype_transport,registrations__buildtype_van,registrations__buildtype_cabrio,registrations__buildtype_stufenheck,registrations__buildtype_kurzheck,registrations__buildtype_other,registrations__buildtype_pickup,registrations__buildtype_coupe,registrations__buildtype_kombi
0,1,1,Country A,296.0,352.0,739.0,471.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,2,Country A,414.0,372.0,904.0,528.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,3,Country A,405.0,604.0,1000.0,953.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,4,Country A,359.0,483.0,828.0,972.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,5,Country A,403.0,474.0,594.0,900.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,8,356,Country A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
356,9,357,Country A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
357,10,358,Country A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
358,11,359,Country A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## select an vehiclegroup

In [8]:
selcted_vehiclegroup = 'vehiclegroup01'

# delete all columns that are for a different country

df1 = df_country_a

# create new dataframe with month and all columns containing vehiclegroup01
df2 = df1.filter(regex=selcted_vehiclegroup)
df2.head()

# drop columns in df1 which contain vehiclegroup
df3 = df1.drop(df1.filter(regex='vehiclegroup').columns, axis=1)
df3.head()

# merge df2 and df3 by index
df4 = df2.merge(df3, left_index=True, right_index=True)
df4.head()

# is there a column month in df4?
df4.columns.str.contains('month').any()
# column month should be the first column, second column is countryname
df4 = df4[['month_normalized','month', 'countryname'] + [col for col in df4.columns if col not in ['month_normalized','month', 'countryname']]]

In [9]:
# drop rows after row 258 (incl. leasing data)
test = df4.drop(df4.index[258:])

## build different dataframes for internal and external data

In [10]:
# dataframe with first 18 columns (internal data)
df_internal = test.iloc[:, :21]
df_internal.head()

,month_normalized,month,countryname,sales_actuals_monthly__vehiclegroup01__orderintake,sales_actuals_monthly__vehiclegroup01__retail,sales_flow_monthly__vehiclegroup01__cp8,sales_stock_monthly__vehiclegroup01__physical,sales_stock_monthly__vehiclegroup01__grossall,sales_stock_monthly__vehiclegroup01__netimporteranddealer,customer_vehicle_relationships__vehiclegroup01__terminated_cars,...,leads__vehiclegroup01__won,leads__vehiclegroup01__first_touch_point,leads__vehiclegroup01__lost,leasing_contracts__vehiclegroup01__end,leasing_contracts__vehiclegroup01__start,premises__vehiclegroup01__flag_order_start,premises__vehiclegroup01__in_sales,premises__vehiclegroup01__flag_production_start,premises__vehiclegroup01__in_production,quota__vehiclegroup01__aq
0,1,1,Country A,296.0,352.0,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,0.0,81.0,NaN,NaN,NaN,NaN,NaN
1,2,2,Country A,414.0,372.0,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,4.0,130.0,NaN,NaN,NaN,NaN,NaN
2,3,3,Country A,405.0,604.0,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,2.0,145.0,NaN,NaN,NaN,NaN,NaN
3,4,4,Country A,359.0,483.0,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,2.0,183.0,NaN,NaN,NaN,NaN,NaN
4,5,5,Country A,403.0,474.0,NaN,NaN,NaN,NaN,10.0,...,NaN,NaN,NaN,4.0,201.0,NaN,NaN,NaN,NaN,NaN


In [11]:
# dataframe with external features
df_external = test.iloc[:, 21:]
df_external.head()

,global_consumer__alcoholic_beverages,global_consumer__apparel,global_consumer__apparel_footwear,global_consumer__appliances,global_consumer__automobiles,global_consumer__average_disposable_income_of_urban_households_nominal,global_consumer__average_disposable_income_of_urban_households_real,global_consumer__average_income,global_consumer__beer,global_consumer__building_materials_garden_supply,...,registrations__buildtype_suv,registrations__buildtype_transport,registrations__buildtype_van,registrations__buildtype_cabrio,registrations__buildtype_stufenheck,registrations__buildtype_kurzheck,registrations__buildtype_other,registrations__buildtype_pickup,registrations__buildtype_coupe,registrations__buildtype_kombi
0,87.206968,255.462236,301.743202,41.505972,230.716125,0.0,0.0,46.394457,34.695874,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,87.206968,255.462236,301.743202,41.505972,230.716125,0.0,0.0,122.084727,34.695874,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,87.206968,255.462236,301.743202,41.505972,230.716125,0.0,0.0,122.084727,34.695874,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,87.206968,255.462236,301.743202,41.505972,230.716125,0.0,0.0,122.084727,34.695874,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,87.206968,255.462236,301.743202,41.505972,230.716125,0.0,0.0,122.084727,34.695874,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# building target values


In [16]:
#take every row from 50 to 255 and 
#X = df_internal.iloc[49:255,:-3]
X = df_internal.iloc[49:255, ~df_internal.columns.str.contains('sales_actuals_monthly__vehiclegroup01__orderintake')]
y = df_internal.iloc[49:255, 3]

X_with_target = df_internal.iloc[49:255,:]


In [17]:
X_with_target

,month_normalized,month,countryname,sales_actuals_monthly__vehiclegroup01__orderintake,sales_actuals_monthly__vehiclegroup01__retail,sales_flow_monthly__vehiclegroup01__cp8,sales_stock_monthly__vehiclegroup01__physical,sales_stock_monthly__vehiclegroup01__grossall,sales_stock_monthly__vehiclegroup01__netimporteranddealer,customer_vehicle_relationships__vehiclegroup01__terminated_cars,...,leads__vehiclegroup01__won,leads__vehiclegroup01__first_touch_point,leads__vehiclegroup01__lost,leasing_contracts__vehiclegroup01__end,leasing_contracts__vehiclegroup01__start,premises__vehiclegroup01__flag_order_start,premises__vehiclegroup01__in_sales,premises__vehiclegroup01__flag_production_start,premises__vehiclegroup01__in_production,quota__vehiclegroup01__aq
49,2,50,Country A,215.0,256.0,NaN,NaN,NaN,NaN,78.0,...,NaN,NaN,NaN,296.0,197.0,0.0,0.0,0.0,0.0,NaN
50,3,51,Country A,250.0,488.0,NaN,NaN,NaN,NaN,79.0,...,NaN,NaN,NaN,376.0,285.0,0.0,0.0,0.0,0.0,NaN
51,4,52,Country A,222.0,552.0,NaN,NaN,NaN,NaN,84.0,...,NaN,NaN,NaN,374.0,346.0,0.0,0.0,200.0,51.0,NaN
52,5,53,Country A,179.0,411.0,NaN,NaN,NaN,NaN,73.0,...,NaN,NaN,NaN,381.0,401.0,0.0,0.0,0.0,51.0,NaN
53,6,54,Country A,209.0,366.0,NaN,NaN,NaN,NaN,143.0,...,NaN,NaN,NaN,313.0,328.0,0.0,0.0,0.0,51.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,11,251,Country A,323.0,355.0,317.0,331.0,311.0,138.0,564.0,...,615.0,218.0,241.0,590.0,432.0,0.0,417.0,0.0,385.0,326.0
251,0,252,Country A,262.0,579.0,305.0,259.0,214.0,180.0,355.0,...,528.0,184.0,195.0,478.0,299.0,0.0,417.0,0.0,385.0,279.0
252,1,253,Country A,266.0,195.0,179.0,272.0,249.0,181.0,353.0,...,445.0,187.0,213.0,279.0,128.0,0.0,417.0,0.0,385.0,138.0
253,2,254,Country A,531.0,247.0,297.0,326.0,357.0,173.0,371.0,...,757.0,267.0,367.0,319.0,158.0,333.0,472.0,0.0,385.0,238.0


## testing wheel

In [22]:
# Include all features to be filled with values
checkup_features = [['quota__vehiclegroup01__aq'],['quota__vehiclegroup01__aq']]

mean_per_month = True
interpolation_linear = True
fill_zero = True

# 
for feature in checkup_features:
    if mean_per_month:
        # add mean per month
        # Fill in missing values using the corresponding month values
        month_means = {}
        for i in range(len(X)):
            month = X.iloc[i]['month_normalized']
            month_values = X.loc[X['month_normalized'] == month, feature[0]]
            if not month_values.isnull().values.all():
                month_means[month] = month_values.mean()

        # Fill in missing values using the corresponding month values
        X[feature[0]+'_meanbymonth']=X[feature[0]].loc[:].fillna(X['month_normalized'].map(month_means))

    if interpolation_linear:
        X[feature[0]+'_interpolated_linear']=X[feature[0]].interpolate(method='linear', limit_direction = 'both')

    if fill_zero:
       X[feature[0]+'_fill_zero']=X[feature[0]].fillna(0)

,month_normalized,month,countryname,sales_actuals_monthly__vehiclegroup01__retail,sales_flow_monthly__vehiclegroup01__cp8,sales_stock_monthly__vehiclegroup01__physical,sales_stock_monthly__vehiclegroup01__grossall,sales_stock_monthly__vehiclegroup01__netimporteranddealer,customer_vehicle_relationships__vehiclegroup01__terminated_cars,customer_vehicle_relationships__vehiclegroup01__active_cars,...,leasing_contracts__vehiclegroup01__end,leasing_contracts__vehiclegroup01__start,premises__vehiclegroup01__flag_order_start,premises__vehiclegroup01__in_sales,premises__vehiclegroup01__flag_production_start,premises__vehiclegroup01__in_production,quota__vehiclegroup01__aq,quota__vehiclegroup01__aq_meanbymonth,quota__vehiclegroup01__aq_interpolated_linear,quota__vehiclegroup01__aq_fill_zero
49,2,50,Country A,256.0,NaN,NaN,NaN,NaN,78.0,141.0,...,296.0,197.0,0.0,0.0,0.0,0.0,NaN,287.375000,83.0,0.0
50,3,51,Country A,488.0,NaN,NaN,NaN,NaN,79.0,163.0,...,376.0,285.0,0.0,0.0,0.0,0.0,NaN,345.000000,83.0,0.0
51,4,52,Country A,552.0,NaN,NaN,NaN,NaN,84.0,152.0,...,374.0,346.0,0.0,0.0,200.0,51.0,NaN,320.600000,83.0,0.0
52,5,53,Country A,411.0,NaN,NaN,NaN,NaN,73.0,165.0,...,381.0,401.0,0.0,0.0,0.0,51.0,NaN,332.866667,83.0,0.0
53,6,54,Country A,366.0,NaN,NaN,NaN,NaN,143.0,272.0,...,313.0,328.0,0.0,0.0,0.0,51.0,NaN,364.800000,83.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,11,251,Country A,355.0,317.0,331.0,311.0,138.0,564.0,633.0,...,590.0,432.0,0.0,417.0,0.0,385.0,326.0,326.000000,326.0,326.0
251,0,252,Country A,579.0,305.0,259.0,214.0,180.0,355.0,539.0,...,478.0,299.0,0.0,417.0,0.0,385.0,279.0,279.000000,279.0,279.0
252,1,253,Country A,195.0,179.0,272.0,249.0,181.0,353.0,449.0,...,279.0,128.0,0.0,417.0,0.0,385.0,138.0,138.000000,138.0,138.0
253,2,254,Country A,247.0,297.0,326.0,357.0,173.0,371.0,673.0,...,319.0,158.0,333.0,472.0,0.0,385.0,238.0,238.000000,238.0,238.0


## SHAP

In [14]:
import shap
import xgboost

model = xgboost.XGBRegressor().fit(X, y)

print('Accuracy Score of the Model is: ',model.score(X, y))

explainer = shap.Explainer(model)
shap_values = explainer(X)

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


ValueError: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, The experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:countryname: object

In [ ]:
shap.summary_plot(shap_values, X)
shap.summary_plot(shap_values, X, plot_type='bar', max_display=5)





In [ ]:
## SHAP
import xgboost
import shap
from sklearn import metrics
import matplotlib

shap.initjs()
X = output_df

countryMapping = {'Country A': 1, 'Country B': 2, 'Country C': 3, 'Country D': 4}
X['countryNo'] = X['countryname'].map(countryMapping)
X = X.drop('countryname', axis=1)

X.dtypes
model = xgboost.XGBRegressor().fit(X, y)

print('Accuracy Score of the Model is: ', model.score(X, y))

explainer = shap.Explainer(model)
# shap_values = explainer(X)
shap_values = explainer.shap_values(X)


